# Задача Машинного перевода
  
Возьмем данные отсюда: [Lots of neat sentence pairs datasets.](http://www.manythings.org/anki/)


In [1]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

Using TensorFlow backend.


In [2]:
num_samples = 10000  # Укажем количество семплов для обучения 
data_path = 'rus.txt' # Путь до файла на диске 

In [3]:
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

In [4]:
# Выведем случайный пример:
print(lines[np.random.randint(0,len(lines))])

Tom said he'd call tomorrow.	Том сказал, что позвонит завтра.


Мы будем решать задачу машинного перевода обученем seq2seq. 

# 1. Предобработка данных  
Закодируем текст в три матрицы `encoder_input_data`, `decoder_input_data`, `decoder_target_data` вида:  
- `encoder_input_data` трехмерный вектор  размерности `(кол-во семплов, максимальная длина предложения, количество символов)` содержащий OhE для английского текста.
- `decoder_input_data` трехмерный вектор  размерности `(кол-во семплов, максимальная длина предложения, количество символов)` содержащий OhE для русского текста.
- `decoder_target_data` это тот же самый массив `decoder_input_data`, но сдвинутый на один символ

In [5]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

In [6]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [7]:
print('Количество семплов', len(input_texts))
print('Количество уникальных токенов в английском тексте:', num_encoder_tokens)
print('Количество уникальных токенов в русском тексте:', num_decoder_tokens)
print('Максимальная длинна последовательности для английского текста:', max_encoder_seq_length)
print('аксимальная длинна последовательности для русского текста:', max_decoder_seq_length)

Количество семплов 10000
Количество уникальных токенов в английском тексте: 72
Количество уникальных токенов в русском тексте: 92
Максимальная длинна последовательности для английского текста: 14
аксимальная длинна последовательности для русского текста: 60


# 2. Создание и обучение модели


<img src='https://miro.medium.com/max/5736/1*1I2tTjCkMHlQ-r73eRn4ZQ.png' width='70%'>  
  
  
  
  
  
Обучим базовую LSTM Seq2Seq модель для предсказания `decoder_target_data` давая на вход `encoder_input_data` и `decoder_input_data`.



In [8]:
batch_size = 64  # Batch size для обучения 
epochs = 100  # Зададим количество эпох
latent_dim = 256  # количество нейронов (размер скрытого слоя закодированного пространства)

## 2.1. Encoder
Создайте `encoder` состоящий из:
- входа размерностью `shape=(None, num_encoder_tokens)`
- LSTM слоя с количеством нейронов `latent_dim` и не забудьте вывести внутренние состояния
- выход: обычный выход и внутренние состояния 

In [9]:
encoder_inputs = # Ваш код здесь
encoder = # Ваш код здесь
encoder_outputs, state_h, state_c = # Ваш код здесь

In [10]:
# Нам не интересен выход с сети, поэтому мы фиксируем только внутренние состояния 
encoder_states = [state_h, state_c]

## 2.2. Decoder
Создадим `decoder`состоящий из:
- входа размерностью `shape=(None, num_decoder_tokens)`
- слоя LSTM с количеством нейронов `latent_dim` возвращающий не только внутренние состояние, но и саму последовательность. Учтите так же то, что начальное состояние должен соответствовать внутренним состояниям из `encoder_states` (Используйте парамтер `initial_state`)
- Каждый выход LSTM свяжите с `Dense` слоем с функцией активации `softmax` для предсказания вектора вероятностей появления каждого символа. 

In [11]:
decoder_inputs = # Ваш код здесь
decoder_lstm = # Ваш код здесь

decoder_outputs, _, _ = # Ваш код здесь

decoder_dense = # Ваш код здесь
decoder_outputs = decoder_dense(decoder_outputs)

## 2.3. Сборка модели
Соберем все кусочки нашей модели

In [12]:
model = # Ваш код здесь

model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 72)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 92)     0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 256), (None, 336896      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  357376      input_2[0][0]                    
                                                                 lstm_1[0][1]                     
          

## 2.4. Обучение модели

In [13]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 13s 2ms/step - loss: 1.0017 - acc: 0.7770 - val_loss: 0.9239 - val_acc: 0.7587
Epoch 2/100
8000/8000 [==============================] - 12s 1ms/step - loss: 0.7151 - acc: 0.8144 - val_loss: 0.7396 - val_acc: 0.7994
Epoch 3/100
8000/8000 [==============================] - 12s 2ms/step - loss: 0.5956 - acc: 0.8407 - val_loss: 0.6582 - val_acc: 0.8162
Epoch 4/100
8000/8000 [==============================] - 13s 2ms/step - loss: 0.5213 - acc: 0.8522 - val_loss: 0.6094 - val_acc: 0.8252
Epoch 5/100
8000/8000 [==============================] - 13s 2ms/step - loss: 0.4784 - acc: 0.8624 - val_loss: 0.5627 - val_acc: 0.8381
Epoch 6/100
8000/8000 [==============================] - 13s 2ms/step - loss: 0.4467 - acc: 0.8702 - val_loss: 0.5455 - val_acc: 0.8428
Epoch 7/100
8000/8000 [==============================] - 13s 2ms/step - loss: 0.4230 - acc: 0.8768 - val_loss: 0.5243 - val_acc: 0.8486


In [14]:
# Сохраним модель
model.save('s2.h5')

/opt/conda/lib/python3.6/site-packages/keras/engine/network.py:888: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


# 3. Inference

Перестроим нашу модель для инференса выполнив следующую последовательность шагов:
1. Задайте модель энкодера Model(), на вход которой пойдет наш обученный `encoder_inputs`, а на выход `encoder_states`), тем саммым мы закодируем ввод и получим начальное состояние декодера
2. Зададим входы для декодера (состояния `h` и `c`)
3. Свяжем выход обученной ранее `decoder_lstm` c `decoder_dense`, заменив инициацию  на состояния `h` и `c`
4. Соберем все вместе в Model()

In [15]:
encoder_model = #Ваш код здесь

decoder_state_input_h = #Ваш код здесь
decoder_state_input_c = #Ваш код здесь

decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = #Ваш код здесь

decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = #Ваш код здесь

Логика очень простая: мы подаем на вход Encoder'a последоватьельность, делаем предсказание, и на его выходе получаем эмбединги (состояния `h` и `c`) всей строки. Затем эти инициации и ранее обученные веса используем для генерации текста.

In [16]:
# Сделаем инверсию наших словарей токенов, чтобы получать на выходе что-то более читаемое на выходе
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

# Напишем функцию для векторизации текста
def vectorizing_custom_word(string = 'some text'):
    input_self_text = string
    input_self_decoding = np.zeros(
        (1, max_encoder_seq_length, num_encoder_tokens),
        dtype='float32')
    for t, char in enumerate(input_self_text):
        input_self_decoding[0, t, input_token_index[char]] = 1.
    input_self_decoding[0, t + 1:, input_token_index[' ']] = 1.
    return input_self_decoding


def decode_sequence(input_seq):
    # Проверка на тип текста 
    if isinstance(input_seq, str):
        input_seq = vectorizing_custom_word(input_seq)
    # Еще бы сделать assert на Русский/английский (наличие токена в словаре)
    
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

Выполним перевод для случайного семпла

In [17]:
ind = np.random.randint(0,num_samples)
input_seq = encoder_input_data[ind:ind+1]
decoded_sentence = decode_sequence(input_seq)
print('Input sentence:', input_texts[ind])
print('Decoded sentence:', decoded_sentence)

Input sentence: Wake up.
Decoded sentence: Просыпайся!



Попробуем "свой" текст: 

In [18]:
custom_string = 'Like a boss!'

In [19]:
decode_sequence(custom_string)

'Поспавитесь за лек.\n'